Experiment Template v0.1.0

# System Information

## Basic Info

In [1]:
%cd /home/apowers/Projects/roc
!date

/home/apowers/Projects/roc
Tue Jun 25 20:50:04 PDT 2024


## GIT

In [2]:
!echo "git hash:" `git rev-parse HEAD`
!echo "git status:"
!git status -s | cat
!echo

git hash: 8899acbffd6dc1b23c150e4ec4e705edadf7323a
git status:
 M poetry.lock
 M pyproject.toml
 M roc/attention.py
 M roc/graphdb.py
 M tests/attention_test.py
?? .fttemplates/
?? ARCHITECTURE.md
?? experiments/
?? roc/feature_extractors/contains.py
?? roc/feature_extractors/ratio.py
?? roc/feature_extractors/shape.py



## Python

In [3]:
!python --version
!pip --version
!echo "installed packages:"
!pip list

Python 3.11.9
pip 24.0 from /home/apowers/Projects/roc/.venv/lib/python3.11/site-packages/pip (python 3.11)
installed packages:
Package                       Version           Editable project location
----------------------------- ----------------- --------------------------
alabaster                     0.7.16
annotated-types               0.7.0
astroid                       3.2.2
asttokens                     2.4.1
attrs                         23.2.0
Babel                         2.15.0
bandit                        1.7.8
black                         24.4.2
cachetools                    5.3.3
certifi                       2024.6.2
cfgv                          3.4.0
chardet                       5.2.0
charset-normalizer            3.3.2
click                         8.1.7
cloudpickle                   3.0.0
colorama                      0.4.6
colored                       2.2.4
comm                          0.2.2
contourpy                     1.2.1
coverage                      7.

## System

In [4]:
!uname -a
!echo
!echo "system cpus:"
!lscpu
!echo
!echo "system memory:"
!lsmem
!free

Linux 77d80cd501c3 5.15.0-78-generic #85-Ubuntu SMP Fri Jul 7 15:25:09 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux

system cpus:
Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         48 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  16
  On-line CPU(s) list:   0-15
Vendor ID:               AuthenticAMD
  Model name:            AMD Ryzen 9 5900HX with Radeon Graphics
    CPU family:          25
    Model:               80
    Thread(s) per core:  2
    Core(s) per socket:  8
    Socket(s):           1
    Stepping:            0
    Frequency boost:     enabled
    CPU max MHz:         3300.0000
    CPU min MHz:         1200.0000
    BogoMIPS:            6587.82
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall n
                         x mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_go
 

# Basic ROC Setup

## Config

In [5]:
!echo ".env file:"
!cat .env
!echo

from roc.config import Config

print("\nconfig:")  # noqa: T201
Config.init()
print(Config.get())  # noqa: T201

.env file:
log_modules="graphdb:INFO;event:INFO"

config:
db_host='127.0.0.1' db_port=7687 db_conn_encrypted=False db_username='' db_password='' db_lazy=False node_cache_size=65536 edge_cache_size=65536 log_enable=True log_level='INFO' log_modules='graphdb:INFO;event:INFO' default_action='pass' perception_components=['delta:perception'] enable_gym_dump_env=False max_dump_frames=10 nethack_spectrum='chars'


## Import Paths

In [6]:
import sys

sys.path.append("./tests")

# Experiment

In [7]:
from helpers.nethack_screens import screens
from helpers.util import StubComponent

from roc.attention import VisionAttention
from roc.component import Component
from roc.feature_extractors.delta import Delta
from roc.feature_extractors.flood import Flood
from roc.feature_extractors.line import Line
from roc.feature_extractors.motion import Motion
from roc.feature_extractors.single import Single
from roc.perception import VisionData


delta = Component.get("delta", "perception")
assert isinstance(delta, Delta)
attention = Component.get("vision", "attention")
assert isinstance(attention, VisionAttention)
flood = Component.get("flood", "perception")
assert isinstance(flood, Flood)
line = Component.get("line", "perception")
assert isinstance(line, Line)
motion = Component.get("motion", "perception")
assert isinstance(motion, Motion)
single = Component.get("single", "perception")
assert isinstance(single, Single)
s = StubComponent(
    input_bus=delta.pb_conn.attached_bus,
    output_bus=attention.att_conn.attached_bus,
)

s.input_conn.send(VisionData(screens[0]["chars"]))
s.input_conn.send(VisionData(screens[1]["chars"]))


# assert attention.saliency_map
print("saliency features", attention.saliency_map.size)  # noqa: T201
print("vision:\n", VisionData(screens[0]["chars"]))  # noqa: T201
print(f"saliency map:\n{attention.saliency_map}")  # noqa: T201
print("saliency max strength", attention.saliency_map.get_max_strength())  # noqa: T201
print("saliency strength (0,0)", attention.saliency_map.get_strength(0, 0))  # noqa: T201
print("saliency strength (16,5)", attention.saliency_map.get_strength(16, 5))  # noqa: T201

2024-06-25 20:50:08.250 | DEBUG    | roc.event:__init__:66 - delta:perception attaching to bus perception
2024-06-25 20:50:08.252 | DEBUG    | roc.event:__init__:66 - vision:attention attaching to bus perception
2024-06-25 20:50:08.252 | DEBUG    | roc.event:__init__:66 - vision:attention attaching to bus attention
2024-06-25 20:50:08.253 | DEBUG    | roc.event:__init__:66 - flood:perception attaching to bus perception
2024-06-25 20:50:08.253 | DEBUG    | roc.event:__init__:66 - line:perception attaching to bus perception
2024-06-25 20:50:08.254 | DEBUG    | roc.event:__init__:66 - motion:perception attaching to bus perception
2024-06-25 20:50:08.254 | DEBUG    | roc.event:__init__:66 - single:perception attaching to bus perception
2024-06-25 20:50:08.255 | DEBUG    | roc.event:__init__:66 - testing-stub:stub attaching to bus attention
2024-06-25 20:50:08.256 | DEBUG    | roc.event:__init__:66 - testing-stub:stub attaching to bus perception


saliency features 9801
vision:
                                                                                
                                                                               
                                                                               
               -----                                                           
               |...|                                                           
               .[@x+                                                           
               |f..|                                                           
               |...|                                                           
               ---|-                                                           
                                                                               
                                                                               
                                                                               
        